In [67]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

In [2]:
data = pd.read_csv("hotel_bookings.csv")

In [6]:
#Feltáró adatelemzés megkezdése
#NÉHÁNY MONDAT EZ MIÉRT FONTOS
#Néhány személyes gondolat amik szerintem a legjobban fogják befolyásolni azt, hogy az adott foglalást le fogják-e mondani

In [7]:
#Minél hamarabb foglalja a vendég a szállást annál nagyobb valószínűséggel fogja lemondani
#Nézzük is meg, hogy a rendelkezésre álló adatok alátámasztják-e a gondolatmenetemet.
pd.pivot_table(data, values = "lead_time", index = "is_canceled") 

,lead_time
is_canceled,
0,79.984687
1,144.848815


In [8]:
#1 azt mutatja, hogy a foglalást lemondták, a 0 pedig azt, hogy nem.
#Mint láthatjuk a lemondott foglalásokat körülbelül kétszer hamarabb foglalták mint azokat amiket nem mondtak le.
#Így levonhatjuk azt a következtetést, hogy minél nagyobb a lead_time-ja egy foglalásnak, annál nagyobb a valőszínűsége, hogy le fogják azt mondani.

In [38]:
#Következőnek azok között a foglalások között érdekelne a lemondási arány amiket az emberek csak hétvégére és csak hétköznapokra foglaltak.
only_week_night = data.loc[data["stays_in_weekend_nights"] == 0, "stays_in_week_nights"].count()
only_weekend_night = data.loc[data["stays_in_week_nights"] == 0, "stays_in_weekend_nights"].count()
only_week_night_no_cancel = data.loc[(data["stays_in_weekend_nights"] == 0) & (data["is_canceled"] == 0), "stays_in_week_nights"].count()
only_weekend_night_no_cancel = data.loc[(data["stays_in_week_nights"] == 0) & (data["is_canceled"] == 0), "stays_in_weekend_nights"].count()
print("Only week night {: .2f} is cancelled".format(1-(only_week_night_no_cancel/only_week_night)))
print("Only weekend night {: .2f} is cancelled".format(1-(only_weekend_night_no_cancel/only_weekend_night)))

Only week night  0.38 is cancelled
Only weekend night  0.25 is cancelled


In [17]:
#13%-kal kevesebb foglalást mondanak le a hétvégéken.
#Valószínűleg azért mert az embereknek a hétköznapokban munka vagy akármilyen más ügyei lehetnek amik miatt le kell mondani a foglalást. Akár egyik napról a másikra is.
#Ez gondot jelent a szállodáknak ugyanis minél később mondja le a foglalását egy vendég annál kevesebb ideje van eladni újra a szobát.
#KÖVETKEZŐNEK FOGOM MEGNÉZNI: Olyan szempontból, hogy hány százalékát mondják le ezeknek a foglalásoknak hirtelen(1-2 nappal az érkezés előtt).
#Az emberek jobban ráérnek a hétvégéken. ÁLTALÁNOS FELVETÉS

In [54]:
canceled_info=data[data["reservation_status"]=="Canceled"]
#Most meg van az összes olyan foglalásunk amiket lemondtak. Ezek közül kell kiválogatni azokat a foglalásokat amiket csak hétköznapokra foglaltak.
canceled_info_week_night = canceled_info[canceled_info["stays_in_weekend_nights"]==0]
#Meg van az összes lemondott hétköznapi foglalás

In [65]:
def month_to_number(string):
    if row["arrival_date_month"] == "January":
        return 1
    elif row["arrival_date_month"] == "February":
        return 2
    elif row["arrival_date_month"] == "March":
        return 3
    elif row["arrival_date_month"] == "April":
        return 4
    elif row["arrival_date_month"] == "May":
        return 5
    elif row["arrival_date_month"] == "June":
        return 6
    elif row["arrival_date_month"] == "July":
        return 7
    elif row["arrival_date_month"] == "August":
        return 8
    elif row["arrival_date_month"] == "September":
        return 9
    elif row["arrival_date_month"] == "October":
        return 10
    elif row["arrival_date_month"] == "November":
        return 11
    else:
        return 12

In [89]:
#Most meg kell számolni az összes olyan foglalást amit legfeljebb 2 nappal az érkezés előtt mondtak le.
less_than_2_count = 0
for index, row in canceled_info_week_night.iterrows():
    arrival_year = row["arrival_date_year"]
    arrival_month = month_to_number(row["arrival_date_month"])
    arrival_day = row["arrival_date_day_of_month"]
    status_change = row["reservation_status_date"].split("-")
    arrival_date = date(arrival_year, arrival_month, arrival_day)
    change_date = date(int(status_change[0]), int(status_change[1]), int(status_change[2]))
    if (arrival_date-change_date).days < 3:
        less_than_2_count+=1
print("A foglalások {: .2f} van legfeljebb 2 nappal érkezés előtt lemondva".format(less_than_2_count/len(canceled_info_week_night)))

A foglalások  0.06 van legfeljebb 2 nappal érkezés előtt lemondva


In [ ]:
#Csupán a foglalások 6%-a van legfeljebb 2 nappal érkezés előtt lemondva.
#Ezáltal az ezekben a szállodákban foglalt vendégeknek csupán töredékének kellett hirtelen lemondani a foglalását.
#Mivel a foglalásoknak kis része tartozik ebbe, nem lehet egyáltalán azt állítani, hogy egy hétköznapi foglalásnál minél közelebb kerülünk az érkezés dátumához annál nagyobb valószínűséggel mondanák le a foglalást.